In [1]:
import os
import numpy as np
import tensorflow
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import datasets, layers, models
import glob
import cv2 as cv
import pdb
import pickle
import ntpath
from copy import deepcopy
import timeit
import cv2 as cv
import os
import numpy as np
import pdb
import ntpath
import glob

In [2]:
import tensorflow as tf
tf.config.list_physical_devices('GPU')

[]

In [3]:
# The paths the user is required to modify in order to predict
saved_model_path = "./model_91.73"
saved_model_task2_andy_path = "./model_andy"
saved_model_task2_louie_path = "./model_louie"
saved_model_task2_ora_path = "./model_ora"
saved_model_task2_tommy_path = "./model_tommy"

task1_predictions_save_path = "./Task1/"
task2_predictions_save_path = "./Task2/"

dataset_test_path = "./Validare"

# Task1

In [4]:
nr_examples = 10000 #number of positive examples per folder of input (write 1e9 for taking the hole folder)

#mandatory that nr_negatives >= nr_examples
nr_negatives = 22 #nr negatives per image
box_size = 96
from_saved_model = True # True is we want to use the model saved at saved_model_path, False if we want to generate it this run

from_saved_model_task2 = True # True is we want to use the saved models for task2, False if we want to generate them


# Paths used to train the model

do_we_train_the_model = False #True if we want to train the model


# Paths below are used only when training (you can ignore them if you don't train the model)
dataset_save_path = "./data/Savefile/"

dataset_andy_path = "./data/andy/"
dataset_andy_truth_path = "./data/andy_annotations.txt"

dataset_louie_path = "./data/louie/"
dataset_louie_truth_path = "./data/louie_annotations.txt"

dataset_ora_path = "./data/ora/"
dataset_ora_truth_path = "./data/ora_annotations.txt"

dataset_tommy_path = "./data/tommy/"
dataset_tommy_truth_path = "./data/tommy_annotations.txt"

valid_faces_character = []

def get_faces(character, nr_examples, images_path, images_annotations_path):
    positive_descriptors = []
    global valid_faces_character
    
    f = open(images_annotations_path, "r")

    ct = 0
    for line in f:
        aux = line.split()
        aux = [aux[0], int(aux[1]), int(aux[2]), int(aux[3]), int(aux[4]), aux[5]]
        
        print(character + ": " + str(ct + 1))

        if ct == nr_examples:
            break
        ct = ct + 1

        image = cv.imread(images_path + aux[0])
        if aux[4] - aux[2] > aux[3] - aux[1]:
            image = image[aux[2] : aux[4], aux[1] : aux[1] + (aux[4] - aux[2])]
        else:
            image = image[aux[2] : aux[2] + (aux[3] - aux[1]), aux[1] : aux[3]]

        global box_size
    
        image = cv.resize(image, (box_size, box_size))

        positive_descriptors.append(image)
        positive_descriptors.append(np.fliplr(image))
        
        valid_faces_character.append(aux[5])
        valid_faces_character.append(aux[5])

    positive_descriptors = np.array(positive_descriptors)
    return positive_descriptors
        
def four_digits_jpg(nr):
    if nr >= 0 and nr <= 9:
        return "000" + str(nr) + ".jpg"
    if nr >= 10 and nr <= 99:
        return "00" + str(nr) + ".jpg"
    if nr >= 100 and nr <= 999:
        return "0" + str(nr) + ".jpg"
    return str(nr) + ".jpg"

def intersection(bbox_a, bbox_b):
    x_a = max(bbox_a[0], bbox_b[0])
    y_a = max(bbox_a[1], bbox_b[1])
    x_b = min(bbox_a[2], bbox_b[2])
    y_b = min(bbox_a[3], bbox_b[3])

    inter_area = max(0, x_b - x_a + 1) * max(0, y_b - y_a + 1)

    return inter_area > 0
    

def find_neg_example(list_of_faces, files, len_f, i, photo_name = None, indices = None, wrong_search_filenames = None):
    img = cv.imread(files[i])
    num_rows = img.shape[0]
    num_cols = img.shape[1]
    ct = 0
    while True:
        ct = ct + 1
        if ct == 100:
            break
        x = np.random.randint(0, num_cols - box_size)
        y = np.random.randint(0, num_rows - box_size)

        ok = True
        if photo_name != None:
            for face in list_of_faces[photo_name]:
                if intersection(face, [x, y, x + box_size, y + box_size]):
                    ok = False
                    break
        else: 
            for face in list_of_faces[four_digits_jpg(i + 1)]:
                if intersection(face, [x, y, x + box_size, y + box_size]):
                    ok = False
                    break

        if ok:
            break
    if ct == 100:
        if photo_name != None:
            aux = np.random.randint(0, len_f - 1)
            return find_neg_example(list_of_faces, files, len_f, aux, wrong_search_filenames[int(indices[aux])], indices, wrong_search_filenames)
        return find_neg_example(list_of_faces, files, len_f, np.random.randint(0, len_f - 1))
    return x, y
    
def get_non_faces(character, nr_examples, nr_negatives, images_path, images_annotations_path):
    negative_descriptors = []
    num_negative_per_image = nr_negatives
    
    f = open(images_annotations_path, "r")
    ct = 0
    list_of_faces = {}
    for line in f:
        aux = line.split()
        aux = [aux[0], int(aux[1]), int(aux[2]), int(aux[3]), int(aux[4]), aux[5]]
        
        if aux[0] == four_digits_jpg(nr_examples + 1):
            break
        if aux[0] in list_of_faces:
            list_of_faces[aux[0]].append([aux[1], aux[2], aux[3], aux[4]])
        else:
            list_of_faces[aux[0]] = [[aux[1], aux[2], aux[3], aux[4]]]
    
    files = glob.glob(images_path + "*.jpg")
    num_images = len(files)
        
    for i in range(num_images):
        if i == nr_examples:
            break
            
        print(character + ": " + str(four_digits_jpg(i + 1)))
        
        img = cv.imread(files[i])
        
        num_rows = img.shape[0]
        num_cols = img.shape[1]
        
        X = []
        Y = []
        
        for j in range(num_negative_per_image):
            x, y = find_neg_example(list_of_faces, files, min(len(files), nr_examples), i)

            X.append(x)
            Y.append(y)

        for idx in range(len(Y)):
            patch = img[Y[idx] : Y[idx] + box_size, X[idx] : X[idx] + box_size]
            negative_descriptors.append(patch)

    negative_descriptors = np.array(negative_descriptors)
    return negative_descriptors
            
    

In [5]:
#Loading or generating positive training data
if do_we_train_the_model:
    if os.path.exists(dataset_save_path + "/valid_faces.npy"):

        valid_faces = np.load(dataset_save_path + "/valid_faces.npy")
        valid_faces_character = np.load(dataset_save_path + "/valid_faces_character.npy")

        print('Am incarcat datele de test pozitive')

    else:
        valid_faces = get_faces("/andy", nr_examples, dataset_andy_path, dataset_andy_truth_path)
        valid_faces = np.concatenate((valid_faces, get_faces("/louie", nr_examples, dataset_louie_path, dataset_louie_truth_path)))
        valid_faces = np.concatenate((valid_faces, get_faces("/ora", nr_examples, dataset_ora_path, dataset_ora_truth_path)))
        valid_faces = np.concatenate((valid_faces, get_faces("/tommy", nr_examples, dataset_tommy_path, dataset_tommy_truth_path)))
        np.save(dataset_save_path + "/valid_faces.npy", valid_faces)
        np.save(dataset_save_path + "/valid_faces_character", valid_faces_character)

    #Loading or generating negative training data
    if os.path.exists(dataset_save_path + "/invalid_faces.npy"):

        invalid_faces = np.load(dataset_save_path + "/invalid_faces.npy")
        print('Am incarcat datele de test negative')

    else:
        invalid_faces = get_non_faces("/andy", nr_examples, nr_negatives, dataset_andy_path, dataset_andy_truth_path)
        invalid_faces = np.concatenate((invalid_faces, get_non_faces("/louie", nr_examples, nr_negatives, dataset_louie_path, dataset_louie_truth_path)))
        invalid_faces = np.concatenate((invalid_faces, get_non_faces("/ora", nr_examples, nr_negatives, dataset_ora_path, dataset_ora_truth_path)))
        invalid_faces = np.concatenate((invalid_faces, get_non_faces("/tommy", nr_examples, nr_negatives, dataset_tommy_path, dataset_tommy_truth_path)))
        np.save(dataset_save_path + "/invalid_faces.npy", invalid_faces)

In [6]:
if do_we_train_the_model:
    #Priting shapes for debug
    print(valid_faces[0].shape)
    print(invalid_faces[0].shape)

    #Merging the positive and negative examples into a dataset
    training_examples = np.concatenate((valid_faces, invalid_faces))
    training_labels = np.concatenate((np.array([np.array(1) for x in range(len(valid_faces))]), \
                                  (np.array([np.array(0) for x in range(len(invalid_faces))]))))

    #Printing final data shapes for debug
    print(training_examples.shape, training_labels.shape)
    print(valid_faces_character[:10])

In [7]:
#The structure of the convolutional neural network

if from_saved_model:
    model = keras.models.load_model(saved_model_path)
    print(f"Modelul a fost incarcat! Este modelul de la {saved_model_path}")
else:
    if not do_we_train_the_model:
        print("Nu exista nici model salvat si nici nu vrem sa antrenam, contradictie!")
    else:
        print("Modelul va fi antrenat!")
        model = keras.models.Sequential([
            keras.layers.Conv2D(filters = 32, kernel_size = (2, 2), activation = 'relu', input_shape = (box_size, box_size, 3)),
            keras.layers.MaxPooling2D(),

            keras.layers.Conv2D(filters = 64, kernel_size = (2, 2), activation = 'relu'),
            keras.layers.MaxPooling2D(),

            keras.layers.Conv2D(filters = 128, kernel_size = (2, 2), activation = 'relu'),
            keras.layers.MaxPooling2D(),

            keras.layers.Flatten(),

            keras.layers.Dense(256, activation = 'relu'),
            keras.layers.Dropout(0.25),

            keras.layers.Dense(64, activation = 'relu'),
            keras.layers.Dropout(0.25),

            keras.layers.Dense(32, activation = 'relu'),
            keras.layers.Dropout(0.25),

            keras.layers.Dense(16, activation = 'relu'),

            keras.layers.Dense(2, activation = 'softmax')
        ])


        opt = keras.optimizers.Adam(learning_rate = 1e-4) 
        model.compile(optimizer = opt, loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

        model.fit(training_examples, training_labels, epochs = 10,shuffle='True')

Modelul a fost incarcat! Este modelul de la ./model_91.73


In [8]:
#Cell used to save the model if it results in a best result!
#model.save('./model_91.73')

In [9]:
#functions imported from Laborator9+10

def intersection_over_union(bbox_a, bbox_b):
    x_a = max(bbox_a[0], bbox_b[0])
    y_a = max(bbox_a[1], bbox_b[1])
    x_b = min(bbox_a[2], bbox_b[2])
    y_b = min(bbox_a[3], bbox_b[3])

    inter_area = max(0, x_b - x_a + 1) * max(0, y_b - y_a + 1)

    box_a_area = (bbox_a[2] - bbox_a[0] + 1) * (bbox_a[3] - bbox_a[1] + 1)
    box_b_area = (bbox_b[2] - bbox_b[0] + 1) * (bbox_b[3] - bbox_b[1] + 1)

    iou = inter_area / float(box_a_area + box_b_area - inter_area)

    return iou

def compute_average_precision(rec, prec):
    # functie adaptata din 2010 Pascal VOC development kit
    m_rec = np.concatenate(([0], rec, [1]))
    m_pre = np.concatenate(([0], prec, [0]))
    for i in range(len(m_pre) - 1, -1, 1):
        m_pre[i] = max(m_pre[i], m_pre[i + 1])
    m_rec = np.array(m_rec)
    i = np.where(m_rec[1:] != m_rec[:-1])[0] + 1
    average_precision = np.sum((m_rec[i] - m_rec[i - 1]) * m_pre[i])
    return average_precision

def non_maximal_suppression(image_detections, image_scores, image_size):
    # xmin, ymin, xmax, ymax
    x_out_of_bounds = np.where(image_detections[:, 2] > image_size[1])[0]
    y_out_of_bounds = np.where(image_detections[:, 3] > image_size[0])[0]
    print(x_out_of_bounds, y_out_of_bounds)
    image_detections[x_out_of_bounds, 2] = image_size[1]
    image_detections[y_out_of_bounds, 3] = image_size[0]
    sorted_indices = np.flipud(np.argsort(image_scores))
    sorted_image_detections = image_detections[sorted_indices]
    sorted_scores = image_scores[sorted_indices]

    is_maximal = np.ones(len(image_detections)).astype(bool)
    iou_threshold = 0.3
    for i in range(len(sorted_image_detections) - 1):
        if is_maximal[i] == True:  # don't change to 'is True' because is a numpy True and is not a python True :)
            for j in range(i + 1, len(sorted_image_detections)):
                if is_maximal[j] == True:  # don't change to 'is True' because is a numpy True and is not a python True :)
                    if intersection_over_union(sorted_image_detections[i],sorted_image_detections[j]) > iou_threshold:is_maximal[j] = False
                    else:  # verificam daca centrul detectiei este in mijlocul detectiei cu scor mai mare
                        c_x = (sorted_image_detections[j][0] + sorted_image_detections[j][2]) / 2
                        c_y = (sorted_image_detections[j][1] + sorted_image_detections[j][3]) / 2
                        if sorted_image_detections[i][0] <= c_x <= sorted_image_detections[i][2] and \
                                sorted_image_detections[i][1] <= c_y <= sorted_image_detections[i][3]:
                            is_maximal[j] = False
    return sorted_image_detections[is_maximal], sorted_scores[is_maximal]


In [10]:
def facial_detector(test_path):
    test_path += "/*.jpg"
    test_files = glob.glob(test_path)
    detections = []
    detection_scores = []
    file_names = []

    for ii in range(len(test_files)):
        start_time = timeit.default_timer()
        patches = []
        squares = []
        img = cv.imread(test_files[ii])

        #We get diverse size patches, resize them to box_size x box_size and adding them to the predict array
        for saiz in range(132, 24,-9):
            for i in range(0, img.shape[0] - saiz, 4):
                for j in range(0, img.shape[1] - saiz, 4):

                    patch = img[i : i + saiz, j : j + saiz, :]
                    patch = cv.resize(patch,(box_size, box_size), interpolation = cv.INTER_AREA)

                    patches.append(patch)
                    squares.append([j, i, j + saiz, i + saiz])

        patches = np.array(patches)
        print(test_files[ii][-8:])

        model_predictions = model.predict(patches)

        predictions = []
        scores = []

        #If the score between 0 and 1 received from the photo is greater than 0.8, we consider it good
        threshold = 0.8
        for i in range(len(patches)):
            if model_predictions[i][1] > threshold:
                predictions.append(squares[i])
                scores.append(model_predictions[i][1])

        nr_preds = len(predictions)
        if nr_preds == 0:
            continue

        predictions, scores = non_maximal_suppression(np.array(predictions), np.array(scores), img.shape)

        print(f"Am extras {len(scores)} predictii din poza {test_files[ii][-8:]}")

        for i in range(len(predictions)):
            file_names.append(test_files[ii][-8:])
            detections.append(predictions[i])
            detection_scores.append(scores[i])
        end_time = timeit.default_timer()
        print(f"Imaginea {test_files[ii][-8:]} s-a procesat in {end_time - start_time}")


    return detections, detection_scores, file_names     

In [11]:
#Generating predictions on the test dataset
detections, scores, file_names = facial_detector(dataset_test_path)

#If the algorithm ran on the dataset at least once, we can use the saved data instead of running it again!
#Decomment the below lines and comment the first (without counting the comment )
# detections = np.load("./data/detections_all_faces.npy")
# scores = np.load("./data/scores_all_faces.npy")
# file_names = np.load("./data/file_names_all_faces.npy")

In [12]:
# Lines that save the generated predictions to the respective paths
np.save(task1_predictions_save_path + "/detections_all_faces.npy", detections)
np.save(task1_predictions_save_path + "/scores_all_faces.npy", scores)
np.save(task1_predictions_save_path + "/file_names_all_faces.npy", file_names)

# Task2

In [13]:
if do_we_train_the_model:
    #Generating the dataset
    task2_andy_training_examples = []
    task2_andy_training_labels = []
    task2_louie_training_examples = []
    task2_louie_training_labels = []
    task2_ora_training_examples = []
    task2_ora_training_labels = []
    task2_tommy_training_examples = []
    task2_tommy_training_labels = []

    for i in range(len(valid_faces)):
        # If we have a photo of character X in dataset, it will represent a positive example for 
        # the respective character classifier and a negative one for the other classifiers
        if valid_faces_character[i] == "andy":

            task2_andy_training_examples.append(valid_faces[i])
            task2_andy_training_labels.append(0)

            task2_louie_training_examples.append(valid_faces[i])
            task2_louie_training_labels.append(1)
            task2_ora_training_examples.append(valid_faces[i])
            task2_ora_training_labels.append(1)
            task2_tommy_training_examples.append(valid_faces[i])
            task2_tommy_training_labels.append(1)

        if valid_faces_character[i] == "louie":

            task2_louie_training_examples.append(valid_faces[i])
            task2_louie_training_labels.append(0)

            task2_andy_training_examples.append(valid_faces[i])
            task2_andy_training_labels.append(1)
            task2_ora_training_examples.append(valid_faces[i])
            task2_ora_training_labels.append(1)
            task2_tommy_training_examples.append(valid_faces[i])
            task2_tommy_training_labels.append(1)
        if valid_faces_character[i] == "ora":

            task2_ora_training_examples.append(valid_faces[i])
            task2_ora_training_labels.append(0)

            task2_louie_training_examples.append(valid_faces[i])
            task2_louie_training_labels.append(1)
            task2_andy_training_examples.append(valid_faces[i])
            task2_andy_training_labels.append(1)
            task2_tommy_training_examples.append(valid_faces[i])
            task2_tommy_training_labels.append(1)
        if valid_faces_character[i] == "tommy":

            task2_tommy_training_examples.append(valid_faces[i])
            task2_tommy_training_labels.append(0)

            task2_louie_training_examples.append(valid_faces[i])
            task2_louie_training_labels.append(1)
            task2_ora_training_examples.append(valid_faces[i])
            task2_ora_training_labels.append(1)
            task2_andy_training_examples.append(valid_faces[i])
            task2_andy_training_labels.append(1)

    task2_andy_training_examples = np.array(task2_andy_training_examples)
    task2_andy_training_labels = np.array(task2_andy_training_labels)
    task2_louie_training_examples = np.array(task2_louie_training_examples)
    task2_louie_training_labels = np.array(task2_louie_training_labels)
    task2_ora_training_examples = np.array(task2_ora_training_examples)
    task2_ora_training_labels = np.array(task2_ora_training_labels)
    task2_tommy_training_examples = np.array(task2_tommy_training_examples)
    task2_tommy_training_labels = np.array(task2_tommy_training_labels)

In [14]:
#For each character we train a classifier that decides whether a photo contains or does not contain that character

#Andy
if from_saved_model_task2:
    
    model_task2_andy = keras.models.load_model(saved_model_task2_andy_path)
    print("Am incarcat modelul pentru Andy")
else:
    
    print("Se antreneaza modelul pentru Andy")
    model_task2_andy = keras.models.Sequential([
        keras.layers.Conv2D(filters = 32, kernel_size = (2, 2), activation = 'relu', input_shape = (box_size, box_size, 3)),
        keras.layers.MaxPooling2D(),

        keras.layers.Conv2D(filters = 64, kernel_size = (2, 2), activation = 'relu'),
        keras.layers.MaxPooling2D(),

        keras.layers.Conv2D(filters = 128, kernel_size = (2, 2), activation = 'relu'),
        keras.layers.MaxPooling2D(),

        keras.layers.Flatten(),

        keras.layers.Dense(256, activation = 'relu'),
        keras.layers.Dropout(0.25),

        keras.layers.Dense(64, activation = 'relu'),
        keras.layers.Dropout(0.25),

        keras.layers.Dense(32, activation = 'relu'),
        keras.layers.Dropout(0.25),

        keras.layers.Dense(16, activation = 'relu'),
        
        keras.layers.Dense(2, activation = 'softmax')
    ])


    opt = keras.optimizers.Adam(learning_rate = 2e-4) 
    model_task2_andy.compile(optimizer = opt, loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

    model_task2_andy.fit(task2_andy_training_examples, task2_andy_training_labels, epochs = 9,shuffle = 'True')

#LOUIE
if from_saved_model_task2:
    
    model_task2_louie = keras.models.load_model(saved_model_task2_louie_path)
    print("Am incarcat modelul pentru Louie")
else:
    
    print("Se antreneaza modelul pentru Louie")
    model_task2_louie = keras.models.Sequential([
        keras.layers.Conv2D(filters = 32, kernel_size = (2, 2), activation = 'relu', input_shape = (box_size, box_size, 3)),
        keras.layers.MaxPooling2D(),

        keras.layers.Conv2D(filters = 64, kernel_size = (2, 2), activation = 'relu'),
        keras.layers.MaxPooling2D(),

        keras.layers.Conv2D(filters = 128, kernel_size = (2, 2), activation = 'relu'),
        keras.layers.MaxPooling2D(),

        keras.layers.Flatten(),

        keras.layers.Dense(256, activation = 'relu'),
        keras.layers.Dropout(0.25),

        keras.layers.Dense(64, activation = 'relu'),
        keras.layers.Dropout(0.25),

        keras.layers.Dense(32, activation = 'relu'),
        keras.layers.Dropout(0.25),

        keras.layers.Dense(16, activation = 'relu'),
        
        keras.layers.Dense(2, activation = 'softmax')
    ])


    opt = keras.optimizers.Adam(learning_rate = 2e-4) 
    model_task2_louie.compile(optimizer = opt, loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

    model_task2_louie.fit(task2_louie_training_examples, task2_louie_training_labels, epochs = 9,shuffle = 'True')

#Ora model
if from_saved_model_task2:
    
    model_task2_ora = keras.models.load_model(saved_model_task2_ora_path)
    print("Am incarcat modelul pentru Ora")
else:
    
    print("Se antreneaza modelul pentru Ora")
    model_task2_ora = keras.models.Sequential([
        keras.layers.Conv2D(filters = 32, kernel_size = (2, 2), activation = 'relu', input_shape = (box_size, box_size, 3)),
        keras.layers.MaxPooling2D(),

        keras.layers.Conv2D(filters = 64, kernel_size = (2, 2), activation = 'relu'),
        keras.layers.MaxPooling2D(),

        keras.layers.Conv2D(filters = 128, kernel_size = (2, 2), activation = 'relu'),
        keras.layers.MaxPooling2D(),

        keras.layers.Flatten(),

        keras.layers.Dense(256, activation = 'relu'),
        keras.layers.Dropout(0.25),

        keras.layers.Dense(64, activation = 'relu'),
        keras.layers.Dropout(0.25),

        keras.layers.Dense(32, activation = 'relu'),
        keras.layers.Dropout(0.25),

        keras.layers.Dense(16, activation = 'relu'),
        
        keras.layers.Dense(2, activation = 'softmax')
    ])


    opt = keras.optimizers.Adam(learning_rate = 2e-4) 
    model_task2_ora.compile(optimizer = opt, loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

    model_task2_ora.fit(task2_ora_training_examples, task2_ora_training_labels, epochs = 9,shuffle = 'True')


#Tommy model
if from_saved_model_task2:
    
    model_task2_tommy = keras.models.load_model(saved_model_task2_tommy_path)
    print("Am incarcat modelul pentru Tommy")
else:
    
    print("Se antreneaza modelul pentru Tommy")
    model_task2_tommy = keras.models.Sequential([
        keras.layers.Conv2D(filters = 32, kernel_size = (2, 2), activation = 'relu', input_shape = (box_size, box_size, 3)),
        keras.layers.MaxPooling2D(),

        keras.layers.Conv2D(filters = 64, kernel_size = (2, 2), activation = 'relu'),
        keras.layers.MaxPooling2D(),

        keras.layers.Conv2D(filters = 128, kernel_size = (2, 2), activation = 'relu'),
        keras.layers.MaxPooling2D(),

        keras.layers.Flatten(),

        keras.layers.Dense(256, activation = 'relu'),
        keras.layers.Dropout(0.25),

        keras.layers.Dense(64, activation = 'relu'),
        keras.layers.Dropout(0.25),

        keras.layers.Dense(32, activation = 'relu'),
        keras.layers.Dropout(0.25),

        keras.layers.Dense(16, activation = 'relu'),
        
        keras.layers.Dense(2, activation = 'softmax')
    ])


    opt = keras.optimizers.Adam(learning_rate = 2e-4) 
    model_task2_tommy.compile(optimizer = opt, loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

    model_task2_tommy.fit(task2_tommy_training_examples, task2_tommy_training_labels, epochs = 9,shuffle='True')

Am incarcat modelul pentru Andy
Am incarcat modelul pentru Louie
Am incarcat modelul pentru Ora
Am incarcat modelul pentru Tommy


In [15]:
#Cell used to save the models!
model_task2_andy.save('./model_andy')
model_task2_louie.save('./model_louie')
model_task2_ora.save('./model_ora')
model_task2_tommy.save('./model_tommy')

INFO:tensorflow:Assets written to: ./model_andy\assets


INFO:tensorflow:Assets written to: ./model_andy\assets


INFO:tensorflow:Assets written to: ./model_louie\assets


INFO:tensorflow:Assets written to: ./model_louie\assets


INFO:tensorflow:Assets written to: ./model_ora\assets


INFO:tensorflow:Assets written to: ./model_ora\assets


INFO:tensorflow:Assets written to: ./model_tommy\assets


INFO:tensorflow:Assets written to: ./model_tommy\assets


In [16]:
def facial_recognition(detections, file_names, path, path_save):
        
    print(f"Analizam {len(detections)} detectii")
    andy_detections = []
    andy_scores = []
    andy_file_names = []

    louie_detections = []
    louie_scores = []
    louie_file_names = []

    ora_detections = []
    ora_scores = []
    ora_file_names = []

    tommy_detections = []
    tommy_scores = []
    tommy_file_names = []


    faces = []

    #We create the list of faces from the task1 detections
    for ii in range(len(detections)):
        patches = []
        squares = []

        img = cv.imread(f"{path}./{file_names[ii]}")
        img = img[int(detections[ii][1]) : int(detections[ii][3]), int(detections[ii][0]) : int(detections[ii][2])]
        img = cv.resize(img, (box_size, box_size), interpolation = cv.INTER_AREA)
        faces.append(img)


    faces = np.array(faces)

    #Predict on the list of faces(scores between 0 and 1 if in the photo is andy/louie/ora/tommy)
    andy_predictions = model_task2_andy.predict(faces)
    louie_predictions = model_task2_louie.predict(faces)
    ora_predictions = model_task2_ora.predict(faces)
    tommy_predictions = model_task2_tommy.predict(faces)

    #Score threshold to consider that the photo contains the face of a character
    threshold = 0.9
    for i in range(len(andy_predictions)):
        
        #Debug info
        if i % 100 == 0:
            print(f"Suntem la detectia {i}: poza {file_names[i]} coordonatele {detections[i]}")

        img = cv.imread(f"{path}./{file_names[i]}")
        img = img[int(detections[i][1]) : int(detections[i][3]), int(detections[i][0]) : int(detections[i][2])]

        #Checking the score for each character if it is within the required threshold 
        if andy_predictions[i][0] >= threshold:
            andy_detections.append(detections[i])
            andy_scores.append(andy_predictions[i][0])
            andy_file_names.append(file_names[i])
            
        if louie_predictions[i][0] >= threshold:
            louie_detections.append(detections[i])
            louie_scores.append(louie_predictions[i][0])
            louie_file_names.append(file_names[i])
            
        if ora_predictions[i][0] >= threshold:
            ora_detections.append(detections[i])
            ora_scores.append(ora_predictions[i][0])
            ora_file_names.append(file_names[i])
            
        if tommy_predictions[i][0] >= threshold:
            tommy_detections.append(detections[i])
            tommy_scores.append(tommy_predictions[i][0])
            tommy_file_names.append(file_names[i])


    andy_detections = np.array(andy_detections)
    andy_scores = np.array(andy_scores)
    andy_file_names = np.array(andy_file_names)

    louie_detections = np.array(louie_detections)
    louie_scores = np.array(louie_scores)
    louie_file_names = np.array(louie_file_names)

    ora_detections = np.array(ora_detections)
    ora_scores = np.array(ora_scores)
    ora_file_names = np.array(ora_file_names)

    tommy_detections = np.array(tommy_detections)
    tommy_scores = np.array(tommy_scores)
    tommy_file_names = np.array(tommy_file_names)
    
    #Saving the predictions
    np.save(path_save + "/detections_andy.npy", andy_detections)
    np.save(path_save + "/scores_andy.npy", andy_scores)
    np.save(path_save + "/file_names_andy.npy", andy_file_names)
    
    np.save(path_save + "/detections_louie.npy", louie_detections)
    np.save(path_save + "/scores_louie.npy", louie_scores)
    np.save(path_save + "/file_names_louie.npy", louie_file_names)
    
    np.save(path_save + "/detections_ora.npy", ora_detections)
    np.save(path_save + "/scores_ora.npy", ora_scores)
    np.save(path_save + "/file_names_ora.npy", ora_file_names)
    
    np.save(path_save + "/detections_tommy.npy", tommy_detections)
    np.save(path_save + "/scores_tommy.npy", tommy_scores)
    np.save(path_save + "/file_names_tommy.npy", tommy_file_names)

In [17]:
facial_recognition(detections, file_names, dataset_test_path, task2_predictions_save_path)

Analizam 1124 detectii
Suntem la detectia 0: poza 0001.jpg coordonatele [184 184 316 316]
Suntem la detectia 100: poza 0020.jpg coordonatele [108 192 141 225]
Suntem la detectia 200: poza 0037.jpg coordonatele [192 196 324 328]
Suntem la detectia 300: poza 0059.jpg coordonatele [  4  88  37 121]
Suntem la detectia 400: poza 0075.jpg coordonatele [236 272 269 305]
Suntem la detectia 500: poza 0099.jpg coordonatele [ 40 136 145 241]
Suntem la detectia 600: poza 0116.jpg coordonatele [308 288 341 321]
Suntem la detectia 700: poza 0135.jpg coordonatele [ 32 156  83 207]
Suntem la detectia 800: poza 0152.jpg coordonatele [416 104 449 137]
Suntem la detectia 900: poza 0169.jpg coordonatele [ 92  72 152 132]
Suntem la detectia 1000: poza 0183.jpg coordonatele [240 188 291 239]
Suntem la detectia 1100: poza 0198.jpg coordonatele [192  88 306 202]
